#  KONBRIEFING


## Importación de librerías.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import country_converter as coco
import matplotlib.pyplot as plt
import re
import hashlib
from urllib.parse import urlparse
import os
import json
from sklearn.feature_extraction.text import TfidfVectorizer


## Importación e información del Dataframe.

In [3]:
df = pd.read_csv("data/KONBRIEFING.csv")

In [4]:
df.head()

,date,title,description,references,date_uploaded
0,April 2024,Facebook page of the local branch of a politic...,"SPÖ Müllendorf - Müllendorf, Burgenland, Austria","[{""title"":""Hackerangriff auf Facebookseite der...",2024-04-29
1,"April 25, 2024",Cyber attack on a construction company in Germany,"Max Wild GmbH - Berkheim, Baden-Württemberg, G...","[{""title"":""Cyberangriff auf Max Wild GmbH"",""ur...",2024-04-29
2,"April 25, 2024",Cyber attack on a city government in France,"Ville de Gravelines - Gravelines, Hauts-de-Fra...","[{""title"":""Notre ville est actuellement victim...",2024-04-29
3,"April 25, 2024",Cyber attack on a traffic management system in...,"KC Scout - Kansas City, Missouri, USA (Jackson...","[{""title"":""KC Scout Alert"",""url"":""https://www....",2024-04-29
4,"April 24, 2024",Mail account of a city government in Belgium h...,"Deinze, Flemish Region, Belgium","[{""title"":""Stadsdiensten van Deinze geplaagd d...",2024-04-29


In [5]:
df.columns

Index(['date', 'title', 'description', 'references', 'date_uploaded'], dtype='object')

A continuación, se incluye una breve explicación sobre la información que aporta cada columna.
- **Date**: Fecha del ataque (aunque algunos valores están incompletos).
- **Title**: Título del incidente.
- **Description**: Descripción del incidente, que puede incluir la víctima o el sector.
- **References**: Fuentes de referencia para el incidente.
- **Date Uploaded**: Fecha de subida del incidente al sistema.

In [6]:
df.describe(include='all')

,date,title,description,references,date_uploaded
count,67310,67326,67358,67358,67358
unique,1129,4027,4030,4053,16
top,"May 31, 2023",Cyber attack on a university of applied scienc...,? - USA,[],2024-04-29
freq,6624,218,1296,2803,4392


In [7]:
df.dtypes

date             object
title            object
description      object
references       object
date_uploaded    object
dtype: object

## Limpieza de datos.
En un análisis de ciberseguridad, las columnas más importantes son **Date** (fecha del ataque), **Title** (título del incidente) y **Description** (descripción del ataque), ya que proporcionan la información necesaria para identificar patrones temporales, tipos de ataques y sectores afectados. Estas son fundamentales para estudios sobre tendencias, como el aumento de incidentes en ciertos periodos o vulnerabilidades en sectores específicos. Por otro lado, las columnas **Date uploaded** y **References** son menos relevantes para el análisis de patrones, ya que solo indican cuándo se documentó el incidente y las fuentes, aunque pueden ser útiles para validar la información.

In [8]:
df.drop(columns=['date_uploaded'], inplace=True)

El proceso consiste en convertir las cadenas de la columna, que pueden representar listas de diccionarios, en estructuras adecuadas para extraer la información relevante.

Primero, las cadenas se convierten en listas de diccionarios, manejando posibles errores de formato mediante excepciones para evitar fallos. Cuando la conversión es exitosa, se extraen los valores de las claves `title` y `url` de los diccionarios dentro de la lista, asignando valores predeterminados como `None` si no se encuentran datos. Esto permite crear dos nuevas columnas: **References title** y **References URL**.

### Manejo de filas duplicadas.
En caso de haber filas duplicadas, deberemos de eliminar estas para que no afecten al análisis posterior.

In [9]:
df.duplicated().sum() # Columnas duplicadas

62970

Los duplicados en el conjunto de datos representan registros que contienen información redundante o repetitiva, lo que puede sesgar los análisis y afectar la calidad de un modelo de Machine Learning. En este caso, se consideran duplicados aquellos registros que comparten el mismo título, descripción y referencias, ya que estas columnas capturan la esencia del contenido del dato. La columna **Date Uploaded**, que refleja el momento en que los datos fueron cargados al sistema, no será relevante para definir duplicados porque no aporta información directa sobre los ciberataques en sí. Al eliminar los duplicados, aseguramos que el modelo trabaje con datos únicos y representativos, evitando sesgos y optimizando la eficiencia del análisis.

In [10]:
df.drop_duplicates(inplace = True)

In [11]:
df.shape

(4388, 4)

### Manejo de filas nulas.

In [12]:
df.isnull().sum()

date           3
title          2
description    0
references     0
dtype: int64

In [13]:
df.dropna(inplace=True)

### Selección de variables categóricas y continuas.
Para realizar correctamente un ánalisis de datos debemos de distinguir entre las variables categóricas, variables que representan diferentes categorías o grupos, y continuas, variables que pueden tomar cualquier valor dentro de un rango determinado.

In [14]:
v_continuas = []
v_categoricas = []
for i in df.columns:
    if df[i].nunique() > 70 or df[i].dtypes in ['float64', 'int64']:
        v_continuas.append(i)
    else:
        v_categoricas.append(i)

print('Variables continuas: {}'.format(', '.join(v_continuas)))
print('Variables categóricas: {}'.format(', '.join(v_categoricas)))

Variables continuas: date, title, description, references
Variables categóricas: 


#### Tratamiento de variables continuas.
Para el modelo de clasificación debemos de pasar todas las columnas a tipo `int`. Por ello, debemos de ver si realizar como en el caso anterior una codificación ordinal o agruparlos.

##### **Date.**

In [15]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [16]:
df.isnull().sum()

date           199
title            0
description      0
references       0
dtype: int64

In [17]:
df.dropna(inplace=True)

In [18]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [19]:
df.head()

,date,title,description,references,year,month,day
0,2024-04-01,Facebook page of the local branch of a politic...,"SPÖ Müllendorf - Müllendorf, Burgenland, Austria","[{""title"":""Hackerangriff auf Facebookseite der...",2024,4,1
1,2024-04-25,Cyber attack on a construction company in Germany,"Max Wild GmbH - Berkheim, Baden-Württemberg, G...","[{""title"":""Cyberangriff auf Max Wild GmbH"",""ur...",2024,4,25
2,2024-04-25,Cyber attack on a city government in France,"Ville de Gravelines - Gravelines, Hauts-de-Fra...","[{""title"":""Notre ville est actuellement victim...",2024,4,25
3,2024-04-25,Cyber attack on a traffic management system in...,"KC Scout - Kansas City, Missouri, USA (Jackson...","[{""title"":""KC Scout Alert"",""url"":""https://www....",2024,4,25
4,2024-04-24,Mail account of a city government in Belgium h...,"Deinze, Flemish Region, Belgium","[{""title"":""Stadsdiensten van Deinze geplaagd d...",2024,4,24


In [20]:
df.drop(columns=['date'], inplace=True)

##### **Description.**
La columna **Description** contiene información clave sobre el lugar donde ocurrió el incidente, proporcionando un contexto geográfico valioso para el análisis. Este dato se puede aprovechar clasificando cada registro según el continente al que pertenece la ubicación de la víctima. Identificar los continentes nos permitirá segmentar los datos de manera más eficiente, ayudando a detectar patrones regionales de ciberataques y sus posibles tendencias.

In [21]:
df['description'].unique()

array(['SPÖ Müllendorf - Müllendorf, Burgenland, Austria',
       'Max Wild GmbH - Berkheim, Baden-Württemberg, Germany (Landkreis Biberach)',
       'Ville de Gravelines - Gravelines, Hauts-de-France, France', ...,
       'Universitat Oberta de Catalunya (UOC) - Barcelona, Catalonia, Spain',
       'Llucmajor, Islas Baleares, Spain',
       'Katholische Kirche Nordharz - Goslar, Lower Saxony, Germany'],
      dtype=object)

In [22]:
df['country'] = df['description'].apply(lambda x: x.split(',')[-1].strip() if isinstance(x, str) else None)

df['country'].unique()[:10]

array(['Austria', 'Germany (Landkreis Biberach)', 'France',
       'Platte County)', 'Belgium', 'Argentina', 'Canada',
       'České noviny (ČN) / Česká tisková kancelář (ČTK) - Czech Republik',
       'Skanlog - Sweden', 'Germany (Landkreis Unterallgäu)'],
      dtype=object)

In [23]:
def clean_country_name(country):
  if '-' in country:
    cleaned_name = country.split('-')[-1].strip()
  else:
    cleaned_name = re.sub(r"\(.*?\)", "", country).strip()
  return cleaned_name.strip()

df['country'] = df['country'].apply(clean_country_name)

In [24]:
df['country'] = df['country'].str.replace(r'\)', '', regex=True)

In [25]:
df['country'].unique()[:10]

array(['Austria', 'Germany', 'France', 'Platte County', 'Belgium',
       'Argentina', 'Canada', 'Czech Republik', 'Sweden', 'Mexico'],
      dtype=object)

In [45]:
continents_mapping = {
    'Africa': [
        'Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cameroon', 'Central African Republic',
        'Chad', 'Comoros', 'Congo', 'Djibouti', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Ethiopia', 'Gabon',
        'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Ivory Coast', 'Kenya', 'Lesotho', 'Liberia', 'Libya', 'Madagascar',
        'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda',
        'Sao Tome and Principe', 'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan',
        'Sudan', 'Tanzania', 'Togo', 'Tunisia', 'Uganda', 'Zambia', 'Zimbabwe', 'Libya / ليبيا'
    ],
    'Asia': [
        'Afghanistan', 'Armenia', 'Bahrain', 'Bangladesh', 'Bhutan', 'Brunei', 'Cambodia', 'China', 'Cyprus',
        'India', 'Indonesia', 'Iran', 'Iraq', 'Israel', 'Japan', 'Jordan', 'Kazakhstan', 'Kuwait', 'Kyrgyzstan',
        'Laos', 'Lebanon', 'Malaysia', 'Maldives', 'Mongolia', 'Myanmar', 'Nepal', 'North Korea', 'Oman', 'Pakistan',
        'Palestine', 'Philippines', 'Qatar', 'Saudi Arabia', 'Singapore', 'South Korea', 'Sri Lanka', 'Syria', 'Taiwan',
        'Tajikistan', 'Thailand', 'Timor-Leste', 'Turkey', 'Turkmenistan', 'United Arab Emirates', 'Uzbekistan', 'Vietnam', 'Yemen',
        "People's Republic of China", 'Republic of China / Taiwan', 'Jordanien / الأردن', 'Malediven / ދިވެހިރާއްޖޭގެ ޖުމްހޫރިއްޔާ'
    ],
    'Europe': [
        'Albania', 'Andorra', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium', 'Bosnia and Herzegovina', 'Bulgaria',
        'Croatia', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Georgia', 'Germany', 'Greece',
        'Hungary', 'Iceland', 'Ireland', 'Italy', 'Kosovo', 'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Malta',
        'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 'North Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania',
        'Russia', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Ukraine', 'United Kingdom',
        'Vatican City', 'Czech Republik', 'Upper Austria', 'Kingdom of the Netherlands', 'Serbien / Србија', 'UK', 'Isle of Man',
        'Vatican', 'Rügen', 'Kreis', 'Schrobenhausen', 'Frankreich', 'Lübbecke', 'Meiningen', 'Bogen', 'Wittgenstein',
        'Zell', 'Ulm', 'Kreis Neuss', 'Koblenz', 'Neckar district', 'Weilburg', 'Grafenau', 'Greifswald', 'Gunzenhausen',
        'Bitterfeld district', 'Bingen district', 'Frankenberg', 'Spessart', 'Jersey', 'Osterzgebirge', 'Hochschwarzwald',
        'Germany ', 'Höchstadt', 'Guernsey', 'Europe', 'Worms', 'Harmony', 'Poly Network', 'PolyNetwork', 'Wormhole'
    ],
    'North America': [
        'Antigua and Barbuda', 'Bahamas', 'Barbados', 'Belize', 'Canada', 'Costa Rica', 'Cuba', 'Dominica', 'Dominican Republic',
        'El Salvador', 'Grenada', 'Guatemala', 'Haiti', 'Honduras', 'Jamaica', 'Mexico', 'Nicaragua', 'Panama',
        'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Vincent and the Grenadines', 'Trinidad and Tobago', 'United States of America (USA)',
        'USA', 'Grönland', 'Puerto Rico', 'Bermuda', 'United States of America', 'Platte County', 'Rockwall County', 'Leelanau County', 'Lehigh County', 'Columbus County', 'DuPage County',
        'DeKalb County', 'Manatee County', 'Miami County', 'Montgomery County', 'Cook County', 'Hawkins County',
        'Williamson County', 'Utah County', 'Rankin County', 'Dade County', 'Roane County', 'Chester County', 'Callaway Country',
        'Howard County', 'Clinton County', 'Marion County', 'Durham County', 'Warren County', 'Peach County', 'Kanawha County',
        'Lexington County', 'Allegan County', 'Sint Maarten', 'Fairfield County', 'Baltimore County', 'Baltimore City', 'Baltimore', 'Bossier Parish',
        'Nicollet County'
    ],
    'South America': [
        'Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela'
    ],
    'Australia': [
        'Australia', 'Fiji', 'Kiribati', 'Marshall Islands', 'Micronesia', 'Nauru', 'New Zealand', 'Palau', 'Papua New Guinea',
        'Samoa', 'Solomon Islands', 'Tonga', 'Tuvalu', 'Vanuatu', 'Oceania'
    ],
    'Africa, Asia, North America, Europe, South America, Australia': ['Global']
}

In [46]:
def assign_continent(country_clean):
  continents = set()

  for continent, countries_list in continents_mapping.items():
    if country_clean in countries_list:
      continents.add(continent)
  return ', '.join(continents) if continents else None

df['continent'] = df['country'].apply(assign_continent)

In [47]:
countries_with_no_continent = df[df['continent'].isnull()]['country'].unique()

# Mostrar la lista de países con 'continent' como None
print("Lista de países con 'continent' como None:")
print(countries_with_no_continent)


Lista de países con 'continent' como None:
['Apex Legends Global Series' 'Pyrmont' 'Pottawatomie' 'KyberSwap'
 'Unibot' 'HTX / Huobi' 'Remitano' 'CoinEx' 'Cloudflare' 'Zunami Protocol'
 'Curve Stablecoin' 'NATO' 'Alphapo' 'Era Lend' 'CoinsPaid'
 'Archive of Our Own' 'Sturdy Finance'
 'Vulnerabilities: Progress Software MOVEit Transfer'
 '0558 targeting of customer email\r\nhttps://msrc.microsoft.com/blog/2023/07/...'
 'KuCoin' 'Kodi' 'Allbridge' 'Poolz' 'Algorand MyAlgo' 'Platypus' 'Azuki'
 'Legacy of War Foundation' 'gb/games/fifa/fifa...' 'Qubit Finance'
 'Compromised accounts of Ukrainian military personnel are being used to attack European government officials involved with refugees from Ukraine.'
 'Trickbot' 'DEUS Finance' 'Ola Finance' 'Bored Ape Yacht Club'
 'Inverse Finance' 'Beanstalk Farms' 'ZEED'
 'How Google Cloud blocked the largest Layer 7 DDoS attack at 46 million rps\r\nhttps://cloud.google.com/blog/products/i...'
 'Crema Finance'
 '399 address that have been sent a mal

In [48]:
df['continent'].isnull().sum()

66

In [50]:
df.dropna(subset=['continent'], inplace=True)

##### **References.**
El proceso consiste en convertir las cadenas de la columna, que pueden representar listas de diccionarios, en estructuras adecuadas para extraer la información relevante.

Primero, las cadenas se convierten en listas de diccionarios, manejando posibles errores de formato mediante excepciones para evitar fallos. Cuando la conversión es exitosa, se extraen los valores de las claves `title` y `url` de los diccionarios dentro de la lista. Esto permite crear dos nuevas columnas: **References title** y **References URL**.

In [51]:
def extract_title_url(reference_column):
    titles, urls = [], []
    for ref in reference_column:
        if not ref:
            titles.append("")
            urls.append("")
            continue
        try:
            ref_json = json.loads(ref)
            if isinstance(ref_json, list) and len(ref_json) > 0:
                first_ref = ref_json[0]
                title = first_ref.get("title", "")
                url = first_ref.get("url", "")
                site_name = urlparse(url).netloc.replace('www.', '') if url else ""
                titles.append(title)
                urls.append(site_name)
            else:
                titles.append("")
                urls.append("")
        except (json.JSONDecodeError, IndexError, TypeError) as e:
            titles.append("")
            urls.append("")

    return titles, urls

In [54]:
df['reference_title'], df['reference_url'] = extract_title_url(df['references'])
df.head(2)

,title,description,references,year,month,day,country,continent,reference_title,reference_url
0,Facebook page of the local branch of a politic...,"SPÖ Müllendorf - Müllendorf, Burgenland, Austria","[{""title"":""Hackerangriff auf Facebookseite der...",2024,4,1,Austria,Europe,Hackerangriff auf Facebookseite der SPÖ Müllen...,meinbezirk.at
1,Cyber attack on a construction company in Germany,"Max Wild GmbH - Berkheim, Baden-Württemberg, G...","[{""title"":""Cyberangriff auf Max Wild GmbH"",""ur...",2024,4,25,Germany,Europe,Cyberangriff auf Max Wild GmbH,maxwild.com


In [55]:
df.drop(columns=['references'], inplace=True)